# Dataset Optimization - Blobs

To reproduce this experiment, first delete or remove the database den_blobs.db that is stored in the results/optimization folder. Otherwise, the TPE optimization will use the 100 trials stored in the existing den_blobs.db and continue with the 101st trial up to the 200th trial. 

In [1]:
import sys  
sys.path.insert(0, '../..')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import optuna
from optuna.samplers import TPESampler
import numpy as np
import pandas as pd

from sklearn.datasets import make_blobs, make_moons
from sklearn.cluster import KMeans, DBSCAN, SpectralClustering, MeanShift, estimate_bandwidth
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import adjusted_mutual_info_score as AMI
from rock import ROCK

import logging
import sys

In [4]:
def getBlobDensities(n_centers):
  return np.round([3/n_centers * x for x in range(1, n_centers + 1)], decimals=2)

In [5]:
def objective(trial):
    seed = 0

    rock_results = []
    other_results = []

    n_samples = trial.suggest_int('n_samples', 1, 16) * 100
    n_features = trial.suggest_int('n_features', 2, 20)
    n_centers = trial.suggest_int('n_centers', 2, 10)

    datasets = []

    for s in range(seed, seed+10):
        datasets.append(make_blobs(n_samples=n_samples, centers=n_centers, n_features=n_features, cluster_std=getBlobDensities(n_centers), random_state=s))
        
    step = 0
    for dataset in datasets: 
        run = {}
        run['step'] = step
        run['n_samples'] = n_samples
        run['n_centers'] = n_centers
        run['n_features'] = n_features
        scaler = StandardScaler()
        data = scaler.fit_transform(dataset[0])
        kmeans = KMeans(n_clusters=n_centers, random_state=seed).fit(data).labels_
        run['k'] = n_centers
        eps, min_pts = (0.2, 2 * n_features)
        dbscan = DBSCAN(eps=eps, min_samples=min_pts).fit(data).labels_
        run['eps'] = eps
        run['min_pts'] = 2 * n_features
        spectral = SpectralClustering(n_clusters=n_centers, random_state=seed).fit(data).labels_
        bandwidth = estimate_bandwidth(data)
        run['bandwidth'] = bandwidth
        mean_shift = MeanShift(bandwidth=bandwidth).fit(data).labels_

        rock = ROCK(tmax=15).fit(data).labels_

        gt = dataset[1]
        rock_results.append(AMI(rock, gt))
        other_results.append(np.max([AMI(kmeans, gt), AMI(dbscan, gt), AMI(spectral, gt), AMI(mean_shift, gt)]))

        run['ROCK'] = AMI(rock, gt)
        run['K_MEANS'] = AMI(kmeans, gt)
        run['DBSCAN'] = AMI(dbscan, gt)
        run['SPECTRAL'] = AMI(spectral, gt)
        run['MEAN_SHIFT'] = AMI(mean_shift, gt)

        step += 1

        experiment.append(run)

    return np.mean(rock_results) - np.mean(other_results)

In [6]:
experiment = []

In [7]:
study_name = 'den_blobs'

In [8]:
storage_name = f'sqlite:///../../results/optimization/{study_name}.db'
sampler = TPESampler(seed=42)
study = optuna.create_study(study_name=study_name, storage=storage_name, sampler=sampler, load_if_exists=True,  direction="maximize")

[I 2021-06-01 15:13:55,777] A new study created in RDB with name: den_blobs


In [9]:
study.optimize(objective, n_trials=100)

[I 2021-06-01 15:14:40,726] Trial 0 finished with value: -0.6082313190701353 and parameters: {'n_samples': 6, 'n_features': 20, 'n_centers': 8}. Best is trial 0 with value: -0.6082313190701353.
[I 2021-06-01 15:15:21,394] Trial 1 finished with value: -0.03773364939895252 and parameters: {'n_samples': 10, 'n_features': 4, 'n_centers': 3}. Best is trial 1 with value: -0.03773364939895252.
[I 2021-06-01 15:15:24,552] Trial 2 finished with value: -0.8526323451498112 and parameters: {'n_samples': 1, 'n_features': 18, 'n_centers': 7}. Best is trial 1 with value: -0.03773364939895252.
[I 2021-06-01 15:16:35,672] Trial 3 finished with value: -0.2194917520058074 and parameters: {'n_samples': 12, 'n_features': 2, 'n_centers': 10}. Best is trial 1 with value: -0.03773364939895252.
[I 2021-06-01 15:17:31,696] Trial 4 finished with value: -0.16328197707720904 and parameters: {'n_samples': 14, 'n_features': 6, 'n_centers': 3}. Best is trial 1 with value: -0.03773364939895252.
[I 2021-06-01 15:17:44,

[I 2021-06-01 15:51:11,604] Trial 39 finished with value: -0.2601294109595216 and parameters: {'n_samples': 5, 'n_features': 8, 'n_centers': 3}. Best is trial 37 with value: 0.0017796584262947945.
[I 2021-06-01 15:52:25,744] Trial 40 finished with value: -0.3380548439321752 and parameters: {'n_samples': 12, 'n_features': 6, 'n_centers': 7}. Best is trial 37 with value: 0.0017796584262947945.
[I 2021-06-01 15:53:24,201] Trial 41 finished with value: 0.0009214663206327245 and parameters: {'n_samples': 14, 'n_features': 10, 'n_centers': 2}. Best is trial 37 with value: 0.0017796584262947945.
[I 2021-06-01 15:54:28,472] Trial 42 finished with value: 0.0009214663206327245 and parameters: {'n_samples': 14, 'n_features': 10, 'n_centers': 2}. Best is trial 37 with value: 0.0017796584262947945.
[I 2021-06-01 15:55:18,111] Trial 43 finished with value: -0.2620591450294322 and parameters: {'n_samples': 14, 'n_features': 9, 'n_centers': 3}. Best is trial 37 with value: 0.0017796584262947945.
[I 20

[I 2021-06-01 16:10:01,996] Trial 79 finished with value: -0.2653784512690106 and parameters: {'n_samples': 4, 'n_features': 3, 'n_centers': 4}. Best is trial 73 with value: 0.033407548327088366.
[I 2021-06-01 16:10:08,589] Trial 80 finished with value: 0.022570992101012877 and parameters: {'n_samples': 2, 'n_features': 2, 'n_centers': 3}. Best is trial 73 with value: 0.033407548327088366.
[I 2021-06-01 16:10:15,156] Trial 81 finished with value: 0.022570992101012877 and parameters: {'n_samples': 2, 'n_features': 2, 'n_centers': 3}. Best is trial 73 with value: 0.033407548327088366.
[I 2021-06-01 16:10:28,277] Trial 82 finished with value: -0.0312123413451616 and parameters: {'n_samples': 4, 'n_features': 3, 'n_centers': 3}. Best is trial 73 with value: 0.033407548327088366.
[I 2021-06-01 16:10:36,154] Trial 83 finished with value: -0.5126161090141399 and parameters: {'n_samples': 2, 'n_features': 4, 'n_centers': 4}. Best is trial 73 with value: 0.033407548327088366.
[I 2021-06-01 16:1

In [10]:
pd.DataFrame(experiment).to_csv('../../results/optimization/den_blobs.csv')